# Football Prediction AI - Analysis & Visualization
## Inspired by 85% Accuracy Tennis Predictions

This notebook provides analysis and visualization of our football prediction model performance.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
sys.path.append('../src')

# Set style
plt.style.use('default')
sns.set_palette("husl")

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## 1. Data Loading and Exploration

In [ ]:
# Load data
try:
    team_stats = pd.read_csv('../data/all_team_stats.csv')
    match_results = pd.read_csv('../data/all_match_results.csv')
    match_features = pd.read_csv('../data/match_features.csv')
    
    print("Data loaded successfully!")
    print(f"Team stats: {len(team_stats)} records")
    print(f"Match results: {len(match_results)} records")
    print(f"Match features: {len(match_features)} records")
except FileNotFoundError as e:
    print(f"Error loading data: {e}")
    print("Please run the data collection scripts first.")

In [ ]:
# Display data info
if 'team_stats' in locals():
    print("Team Stats Overview:")
    display(team_stats.head())
    print("\nTeam Stats Info:")
    print(team_stats.info())

## 2. Team Performance Analysis

In [ ]:
if 'team_stats' in locals():
    # Create team performance visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Points distribution
    axes[0, 0].hist(team_stats['points'], bins=15, alpha=0.7, edgecolor='black')
    axes[0, 0].set_title('Points Distribution')
    axes[0, 0].set_xlabel('Points')
    axes[0, 0].set_ylabel('Frequency')
    
    # Goal difference vs Points
    axes[0, 1].scatter(team_stats['goal_difference'], team_stats['points'], alpha=0.7)
    axes[0, 1].set_title('Goal Difference vs Points')
    axes[0, 1].set_xlabel('Goal Difference')
    axes[0, 1].set_ylabel('Points')
    
    # Win percentage distribution
    win_pct = team_stats['wins'] / team_stats['matches_played'] * 100
    axes[1, 0].hist(win_pct, bins=15, alpha=0.7, edgecolor='black')
    axes[1, 0].set_title('Win Percentage Distribution')
    axes[1, 0].set_xlabel('Win Percentage (%)')
    axes[1, 0].set_ylabel('Frequency')
    
    # Top teams by points
    top_teams = team_stats.nlargest(10, 'points')
    axes[1, 1].barh(range(len(top_teams)), top_teams['points'])
    axes[1, 1].set_yticks(range(len(top_teams)))
    axes[1, 1].set_yticklabels(top_teams['team_name'])
    axes[1, 1].set_title('Top 10 Teams by Points')
    axes[1, 1].set_xlabel('Points')
    
    plt.tight_layout()
    plt.show()

## 3. Match Results Analysis

In [ ]:
if 'match_results' in locals():
    # Result distribution
    result_counts = match_results['result'].value_counts()
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Pie chart of results
    axes[0].pie(result_counts.values, labels=['Home Win', 'Draw', 'Away Win'], autopct='%1.1f%%')
    axes[0].set_title('Match Result Distribution')
    
    # Goals scored distribution
    axes[1].hist(match_results['total_goals'], bins=range(0, 10), alpha=0.7, edgecolor='black')
    axes[1].set_title('Total Goals per Match Distribution')
    axes[1].set_xlabel('Total Goals')
    axes[1].set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()
    
    print("\nMatch Statistics:")
    print(f"Average goals per match: {match_results['total_goals'].mean():.2f}")
    print(f"Home win percentage: {(result_counts['H'] / len(match_results) * 100):.1f}%")
    print(f"Draw percentage: {(result_counts['D'] / len(match_results) * 100):.1f}%")
    print(f"Away win percentage: {(result_counts['A'] / len(match_results) * 100):.1f}%")

## 4. Feature Correlation Analysis

In [ ]:
if 'match_features' in locals():
    # Select numeric features for correlation
    numeric_features = match_features.select_dtypes(include=[np.number]).columns
    feature_subset = [col for col in numeric_features if not col.startswith('target_')]
    
    # Calculate correlation matrix
    correlation_matrix = match_features[feature_subset].corr()
    
    # Plot correlation heatmap
    plt.figure(figsize=(20, 16))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
                square=True, fmt='.2f', cbar_kws={'shrink': 0.8})
    plt.title('Feature Correlation Matrix', fontsize=16)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

## 5. Model Performance Analysis

In [ ]:
# Model performance comparison (you'll need to run this after training models)
# This is example data - replace with actual model results
model_performance = {
    'Random Forest': 0.72,
    'Gradient Boosting': 0.75,
    'XGBoost': 0.78,
    'LightGBM': 0.76,
    'XGBoost Optimized': 0.82
}

# Create performance visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Bar chart of model accuracies
models = list(model_performance.keys())
accuracies = list(model_performance.values())

bars = axes[0].bar(models, accuracies, color=['skyblue', 'lightgreen', 'orange', 'pink', 'red'])
axes[0].set_title('Model Performance Comparison')
axes[0].set_ylabel('Accuracy')
axes[0].set_ylim(0, 1)
axes[0].axhline(y=0.85, color='red', linestyle='--', label='Target (85%)')

# Add value labels on bars
for bar, acc in zip(bars, accuracies):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                f'{acc:.1%}', ha='center', va='bottom')

axes[0].legend()
axes[0].tick_params(axis='x', rotation=45)

# Progress toward target
target = 0.85
best_model = max(model_performance, key=model_performance.get)
best_accuracy = model_performance[best_model]

progress = min(best_accuracy / target, 1.0)
axes[1].pie([progress, 1-progress], labels=[f'Achieved\n{best_accuracy:.1%}', f'Remaining\n{max(0, target-best_accuracy):.1%}'],
           colors=['green', 'lightgray'], startangle=90)
axes[1].set_title(f'Progress Toward 85% Target\n(Best: {best_model})')

plt.tight_layout()
plt.show()

print(f"\nBest Model: {best_model}")
print(f"Best Accuracy: {best_accuracy:.2%}")
print(f"Target Accuracy: 85%")
print(f"Progress: {'✅ TARGET ACHIEVED!' if best_accuracy >= 0.85 else f'Need {(0.85 - best_accuracy)*100:.1f} more percentage points'}")

## 6. Feature Importance Analysis

In [ ]:
# Example feature importance (replace with actual model feature importance)
feature_importance = {
    'form_difference': 0.15,
    'points_per_game_difference': 0.12,
    'home_recent_form_points': 0.10,
    'goal_difference_difference': 0.09,
    'away_recent_form_points': 0.08,
    'home_win_percentage': 0.07,
    'away_win_percentage': 0.06,
    'h2h_home_wins': 0.05,
    'possession_difference': 0.04,
    'home_advantage': 0.03
}

# Plot feature importance
features = list(feature_importance.keys())
importance = list(feature_importance.values())

plt.figure(figsize=(12, 8))
plt.barh(range(len(features)), importance, color='steelblue')
plt.yticks(range(len(features)), features)
plt.xlabel('Feature Importance')
plt.title('Top 10 Most Important Features for Match Prediction')
plt.gca().invert_yaxis()

# Add value labels
for i, v in enumerate(importance):
    plt.text(v + 0.001, i, f'{v:.3f}', va='center')

plt.tight_layout()
plt.show()

## 7. Interactive World Cup Bracket Visualization

In [ ]:
# Create an interactive tournament bracket visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Example World Cup bracket data
bracket_data = {
    'Round of 16': ['Argentina', 'Australia', 'France', 'Denmark', 'Brazil', 'Switzerland', 'Belgium', 'Croatia',
                    'Poland', 'USA', 'Spain', 'Japan', 'Portugal', 'Morocco', 'England', 'Netherlands'],
    'Quarter-finals': ['Argentina', 'France', 'Brazil', 'Croatia', 'USA', 'Spain', 'Morocco', 'England'],
    'Semi-finals': ['Argentina', 'Croatia', 'Spain', 'England'],
    'Final': ['Argentina', 'Spain'],
    'Winner': ['Argentina']
}

# Create bracket visualization
fig = go.Figure()

# Add teams for each round
rounds = ['Round of 16', 'Quarter-finals', 'Semi-finals', 'Final', 'Winner']
colors = ['lightblue', 'lightgreen', 'yellow', 'orange', 'gold']

for i, (round_name, teams) in enumerate(bracket_data.items()):
    y_positions = np.linspace(0, len(teams)-1, len(teams))
    
    fig.add_trace(go.Scatter(
        x=[i] * len(teams),
        y=y_positions,
        mode='markers+text',
        marker=dict(size=20, color=colors[i]),
        text=teams,
        textposition="middle center",
        name=round_name,
        showlegend=True
    ))

fig.update_layout(
    title='World Cup Prediction Bracket',
    xaxis=dict(tickmode='array', tickvals=list(range(5)), ticktext=rounds),
    yaxis=dict(showticklabels=False),
    width=1000,
    height=600
)

fig.show()

## 8. Confidence Analysis

In [ ]:
# Example confidence distribution (replace with actual prediction confidence data)
confidence_data = np.random.beta(2, 2, 1000) * 0.4 + 0.4  # Simulated confidence scores

plt.figure(figsize=(12, 8))

# Confidence distribution
plt.subplot(2, 2, 1)
plt.hist(confidence_data, bins=30, alpha=0.7, edgecolor='black')
plt.axvline(np.mean(confidence_data), color='red', linestyle='--', label=f'Mean: {np.mean(confidence_data):.2%}')
plt.xlabel('Prediction Confidence')
plt.ylabel('Frequency')
plt.title('Prediction Confidence Distribution')
plt.legend()

# Confidence vs Accuracy (simulated)
plt.subplot(2, 2, 2)
confidence_bins = np.linspace(0.4, 0.9, 6)
accuracy_by_confidence = [0.65, 0.70, 0.75, 0.80, 0.85]  # Simulated accuracy by confidence
plt.plot(confidence_bins[:-1], accuracy_by_confidence, 'o-', linewidth=2, markersize=8)
plt.xlabel('Confidence Level')
plt.ylabel('Actual Accuracy')
plt.title('Accuracy vs Confidence Level')
plt.grid(True, alpha=0.3)

# High confidence predictions
plt.subplot(2, 2, 3)
high_conf_mask = confidence_data > 0.7
high_conf_pct = np.sum(high_conf_mask) / len(confidence_data) * 100
plt.pie([high_conf_pct, 100-high_conf_pct], labels=[f'High Confidence\n(>70%): {high_conf_pct:.1f}%', 
                                                   f'Lower Confidence\n(≤70%): {100-high_conf_pct:.1f}%'],
        colors=['green', 'lightgray'], startangle=90)
plt.title('Prediction Confidence Distribution')

# Confidence over time (simulated)
plt.subplot(2, 2, 4)
time_points = range(50)
confidence_trend = 0.6 + 0.2 * np.sin(np.linspace(0, 4*np.pi, 50)) + np.random.normal(0, 0.02, 50)
plt.plot(time_points, confidence_trend, linewidth=2)
plt.xlabel('Match Number')
plt.ylabel('Average Confidence')
plt.title('Confidence Trend Over Time')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Average Prediction Confidence: {np.mean(confidence_data):.2%}")
print(f"High Confidence Predictions (>70%): {high_conf_pct:.1f}%")
print(f"Target Confidence for 85% Accuracy: 75%+")

## 9. Summary and Next Steps

In [ ]:
print("🏆 FOOTBALL PREDICTION AI ANALYSIS SUMMARY")
print("="*50)
print("Inspired by 85% accuracy tennis predictions")
print()
print("Key Insights:")
print("• Form difference is the most important predictive feature")
print("• XGBoost with optimization performs best")
print("• Home advantage factor contributes significantly")
print("• Head-to-head statistics provide valuable context")
print()
print("Current Status:")
print(f"• Best Model Accuracy: {max(model_performance.values()):.2%}")
print(f"• Target Accuracy: 85%")
print(f"• Status: {'🎉 TARGET ACHIEVED!' if max(model_performance.values()) >= 0.85 else '🔄 IN PROGRESS'}")
print()
print("Next Steps for Improvement:")
print("1. Collect more comprehensive data (injuries, weather, etc.)")
print("2. Add player-level statistics and formations")
print("3. Implement ensemble methods")
print("4. Add time-decay factors for historical data")
print("5. Include bookmaker odds as features")
print("6. Validate on real World Cup matches")